In [ ]:
! pip install google-cloud-aiplatform[adk,agent_engines] google-auth google-genai httpx beautifulsoup4 --quiet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 95.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have pro

In [ ]:
# @title Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
# @title Setup Project and Vertex AI
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

STAGING_BUCKET = "gs://dsf345gjt"

# Initialize the Vertex AI SDK
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

MODEL = "gemini-2.5-flash"

qwiklabs-gcp-01-e87a8593b193


In [ ]:
# @title These ENV variables are used by Gemini Later
import os

# Tell google-genai to use Vertex AI and where
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [ ]:
# @title Set BigQuery Variables
DATASET_NAME = "fda_dataset"
DOWNLOAD_URL = "https://fdc.nal.usda.gov/fdc-datasets/FoodData_Central_foundation_food_csv_2025-04-24.zip"
EXTRACT_DIR = "fooddata_csv"

In [ ]:
# @title Download and Unzip Data File
import os
import requests
import zipfile
import shutil

# --- Config ---
url = DOWNLOAD_URL
zip_filename = "fooddata.zip"
extract_dir = EXTRACT_DIR  # expects EXTRACT_DIR to be defined in a previous cell

# --- Always start from scratch ---
if os.path.exists(zip_filename):
    try:
        os.remove(zip_filename)
        print(f"Removed existing zip: {zip_filename}")
    except Exception as e:
        print(f"Warning: could not remove existing zip ({zip_filename}): {e}")

if os.path.isdir(extract_dir):
    try:
        shutil.rmtree(extract_dir)
        print(f"Removed existing directory: {extract_dir}")
    except Exception as e:
        print(f"Warning: could not remove existing directory ({extract_dir}): {e}")

# --- Download fresh zip ---
print("Downloading dataset...")
with requests.get(url, stream=True, timeout=60) as response:
    response.raise_for_status()
    with open(zip_filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1 MB chunks
            if chunk:
                f.write(chunk)
print("Download complete.")

# --- Extract into a fresh directory ---
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, "r") as zip_ref:
    zip_ref.extractall(extract_dir)
print("Extraction complete.")

# --- List all extracted files ---
print("Extracted files:")
for root, dirs, files in os.walk(extract_dir):
    for filename in sorted(files):
        print(os.path.join(root, filename))


Removed existing zip: fooddata.zip
Removed existing directory: fooddata_csv
Download complete.
Extraction complete.
Extracted files:
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/Download API Field Descriptions.xlsx
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/acquisition_samples.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/agricultural_samples.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food_attribute.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food_attribute_type.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food_calorie_conversion_factor.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food_category.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food_component.csv
fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food_nutrient.csv
fooddata_csv/FoodData_Central

In [ ]:
# @title Save CSV files to BigQuery
import os
import glob
from google.cloud import bigquery
from google.api_core.exceptions import NotFound, GoogleAPICallError, BadRequest

# Where you extracted the USDA CSVs (expects EXTRACT_DIR set earlier)
BASE_DIR = EXTRACT_DIR
MAX_BAD_RECORDS = 100  # <-- allow up to 100 problematic rows per file

# Try to find the inner folder automatically (e.g., FoodData_Central_foundation_food_csv_2025-04-24)
subdirs = [d for d in glob.glob(os.path.join(BASE_DIR, "*")) if os.path.isdir(d)]
DATA_DIR = subdirs[0] if subdirs else BASE_DIR

print("Using data directory:", DATA_DIR)

client = bigquery.Client(project=PROJECT_ID)
dataset_ref = bigquery.DatasetReference(PROJECT_ID, DATASET_NAME)

# Ensure the dataset exists (create if it doesn't)
try:
    client.get_dataset(dataset_ref)
    print(f"Dataset {PROJECT_ID}.{DATASET_NAME} exists.")
except NotFound:
    ds = bigquery.Dataset(dataset_ref)
    ds.location = "US"  # change if needed
    client.create_dataset(ds)
    print(f"Created dataset {PROJECT_ID}.{DATASET_NAME} in location {ds.location}.")

# Load config for CSVs
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    autodetect=True,                         # Let BigQuery infer schema
    skip_leading_rows=1,                     # Header row
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    field_delimiter=",",
    quote_character='"',
    allow_quoted_newlines=True,
    encoding="UTF-8",
    max_bad_records=MAX_BAD_RECORDS,         # <-- tolerate up to N bad rows
    ignore_unknown_values=True,            # (Optional) tolerate extra columns
)

# Iterate over all CSV files and load each into a table named after the file (minus .csv)
csv_paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.csv")))
if not csv_paths:
    raise FileNotFoundError(f"No CSV files found in {DATA_DIR}")

summary = []
failures = []

for csv_path in csv_paths:
    table_name = os.path.splitext(os.path.basename(csv_path))[0]
    table_ref = dataset_ref.table(table_name)
    print(f"\nLoading {csv_path} -> {PROJECT_ID}.{DATASET_NAME}.{table_name}")

    try:
        with open(csv_path, "rb") as f:
            load_job = client.load_table_from_file(f, table_ref, job_config=job_config)

        # Wait for completion; raises on job failure
        load_job.result()

        # Job may still have partial errors (within max_bad_records)
        if load_job.errors:
            print(f"Completed with {len(load_job.errors)} load errors (tolerated by max_bad_records={MAX_BAD_RECORDS}).")
            for err in load_job.errors[:5]:
                print("  -", err)

        table = client.get_table(table_ref)
        print(f"Loaded {table.num_rows} rows into {table.full_table_id}")
        summary.append((table_name, table.num_rows))

    except (BadRequest, GoogleAPICallError, Exception) as e:
        # Try to surface job-level error details if available
        err_details = None
        try:
            if 'load_job' in locals() and load_job is not None:
                if getattr(load_job, "error_result", None):
                    err_details = load_job.error_result
                elif getattr(load_job, "errors", None):
                    err_details = load_job.errors[:5]
        except Exception:
            pass

        print(f"ERROR loading {table_name}: {e}")
        if err_details:
            print("  Details:", err_details)
        failures.append((table_name, str(e)))

print("\n=== Load Summary ===")
for name, rows in summary:
    print(f"{name}: {rows} rows")

if failures:
    print("\n=== Failures ===")
    for name, msg in failures:
        print(f"{name}: {msg}")
else:
    print("\nNo failures 🎉")


Using data directory: fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24
Dataset qwiklabs-gcp-01-e87a8593b193.fda_dataset exists.

Loading fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/acquisition_samples.csv -> qwiklabs-gcp-01-e87a8593b193.fda_dataset.acquisition_samples
Loaded 7999 rows into qwiklabs-gcp-01-e87a8593b193:fda_dataset.acquisition_samples

Loading fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/agricultural_samples.csv -> qwiklabs-gcp-01-e87a8593b193.fda_dataset.agricultural_samples
Loaded 810 rows into qwiklabs-gcp-01-e87a8593b193:fda_dataset.agricultural_samples

Loading fooddata_csv/FoodData_Central_foundation_food_csv_2025-04-24/food.csv -> qwiklabs-gcp-01-e87a8593b193.fda_dataset.food
Completed with 80 load errors (tolerated by max_bad_records=100).
  - {'reason': 'invalid', 'message': 'Error while reading data, error message: Failed to parse input string "7/19/2023"; line_number: 55226 byte_offset_to_start_of_line: 4599840 colu

In [ ]:
# @title Authenticate to BQ

import google.auth
from google.cloud import bigquery
creds, proj = google.auth.default()
print("ADC project:", proj)

# This just tests to ensure we are authenticated to BQ
bq = bigquery.Client(project=PROJECT_ID, credentials=creds)
bq.query("SELECT 1").result()
print("BigQuery query OK")

ADC project: qwiklabs-gcp-01-e87a8593b193
BigQuery query OK


In [ ]:
# @title This just tests to ensure we are authenticate with Gemini.

# Quick sanity check using google-genai directly:
from google import genai
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
resp = client.models.generate_content(model="gemini-2.5-flash", contents="hello")
print(resp.text)


Hello there! How can I help you today?


In [ ]:
# @title ADK Imports
import asyncio
import os
import google.auth
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.bigquery import BigQueryCredentialsConfig, BigQueryToolset
from google.adk.tools.bigquery.config import BigQueryToolConfig, WriteMode
from google.adk.tools import google_search   # built-in Google Search tool


In [ ]:
# @title Build the BigQuery Agent. Uses the BigQuery tool to access the BQ Dataset created earlier


# Uses Application Default Credentials for BigQuery (gcloud or service account).
adc, _ = google.auth.default()
bq_credentials = BigQueryCredentialsConfig(credentials=adc)

# Read-only tool config (blocks DDL/DML). You can change to WriteMode.ALLOWED later if needed.
bq_tool_cfg = BigQueryToolConfig(write_mode=WriteMode.BLOCKED)

# Instantiate the BigQuery toolset
bq_tools = BigQueryToolset(
    credentials_config=bq_credentials,
    bigquery_tool_config=bq_tool_cfg
)

DB_SCHEMA = """

[{
  "table_name": "lab_method_nutrient",
  "fields": [{
    "column_name": "lab_method_id",
    "data_type": "INT64"
  }, {
    "column_name": "nutrient_id",
    "data_type": "INT64"
  }]
}, {
  "table_name": "food_update_log_entry",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }, {
    "column_name": "last_updated",
    "data_type": "DATE"
  }]
}, {
  "table_name": "input_food",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_of_input_food",
    "data_type": "INT64"
  }, {
    "column_name": "seq_num",
    "data_type": "STRING"
  }, {
    "column_name": "amount",
    "data_type": "STRING"
  }, {
    "column_name": "ingredient_code",
    "data_type": "STRING"
  }, {
    "column_name": "ingredient_description",
    "data_type": "STRING"
  }, {
    "column_name": "unit",
    "data_type": "STRING"
  }, {
    "column_name": "portion_code",
    "data_type": "STRING"
  }, {
    "column_name": "portion_description",
    "data_type": "STRING"
  }, {
    "column_name": "gram_weight",
    "data_type": "STRING"
  }, {
    "column_name": "retention_code",
    "data_type": "STRING"
  }]
}, {
  "table_name": "sub_sample_food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id_of_sample_food",
    "data_type": "INT64"
  }]
}, {
  "table_name": "food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "data_type",
    "data_type": "STRING"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }, {
    "column_name": "food_category_id",
    "data_type": "INT64"
  }, {
    "column_name": "publication_date",
    "data_type": "DATE"
  }]
}, {
  "table_name": "foundation_food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "NDB_number",
    "data_type": "INT64"
  }, {
    "column_name": "footnote",
    "data_type": "STRING"
  }]
}, {
  "table_name": "market_acquisition",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "brand_description",
    "data_type": "STRING"
  }, {
    "column_name": "expiration_date",
    "data_type": "DATE"
  }, {
    "column_name": "label_weight",
    "data_type": "STRING"
  }, {
    "column_name": "location",
    "data_type": "STRING"
  }, {
    "column_name": "acquisition_date",
    "data_type": "DATE"
  }, {
    "column_name": "sales_type",
    "data_type": "STRING"
  }, {
    "column_name": "sample_lot_nbr",
    "data_type": "STRING"
  }, {
    "column_name": "sell_by_date",
    "data_type": "DATE"
  }, {
    "column_name": "store_city",
    "data_type": "STRING"
  }, {
    "column_name": "store_name",
    "data_type": "STRING"
  }, {
    "column_name": "store_state",
    "data_type": "STRING"
  }, {
    "column_name": "upc_code",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_protein_conversion_factor",
  "fields": [{
    "column_name": "food_nutrient_conversion_factor_id",
    "data_type": "INT64"
  }, {
    "column_name": "value",
    "data_type": "FLOAT64"
  }]
}, {
  "table_name": "lab_method_code",
  "fields": [{
    "column_name": "lab_method_id",
    "data_type": "INT64"
  }, {
    "column_name": "code",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_nutrient_conversion_factor",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }]
}, {
  "table_name": "food_calorie_conversion_factor",
  "fields": [{
    "column_name": "food_nutrient_conversion_factor_id",
    "data_type": "INT64"
  }, {
    "column_name": "protein_value",
    "data_type": "FLOAT64"
  }, {
    "column_name": "fat_value",
    "data_type": "FLOAT64"
  }, {
    "column_name": "carbohydrate_value",
    "data_type": "FLOAT64"
  }]
}, {
  "table_name": "food_portion",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "seq_num",
    "data_type": "INT64"
  }, {
    "column_name": "amount",
    "data_type": "FLOAT64"
  }, {
    "column_name": "measure_unit_id",
    "data_type": "INT64"
  }, {
    "column_name": "portion_description",
    "data_type": "STRING"
  }, {
    "column_name": "modifier",
    "data_type": "STRING"
  }, {
    "column_name": "gram_weight",
    "data_type": "FLOAT64"
  }, {
    "column_name": "data_points",
    "data_type": "INT64"
  }, {
    "column_name": "footnote",
    "data_type": "STRING"
  }, {
    "column_name": "min_year_acquired",
    "data_type": "INT64"
  }]
}, {
  "table_name": "sample_food",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }]
}, {
  "table_name": "lab_method",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }, {
    "column_name": "technique",
    "data_type": "STRING"
  }]
}, {
  "table_name": "nutrient",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "unit_name",
    "data_type": "STRING"
  }, {
    "column_name": "nutrient_nbr",
    "data_type": "FLOAT64"
  }, {
    "column_name": "rank",
    "data_type": "FLOAT64"
  }]
}, {
  "table_name": "food_nutrient",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "nutrient_id",
    "data_type": "INT64"
  }, {
    "column_name": "amount",
    "data_type": "FLOAT64"
  }, {
    "column_name": "data_points",
    "data_type": "INT64"
  }, {
    "column_name": "derivation_id",
    "data_type": "INT64"
  }, {
    "column_name": "min",
    "data_type": "FLOAT64"
  }, {
    "column_name": "max",
    "data_type": "FLOAT64"
  }, {
    "column_name": "median",
    "data_type": "FLOAT64"
  }, {
    "column_name": "footnote",
    "data_type": "STRING"
  }, {
    "column_name": "min_year_acquired",
    "data_type": "INT64"
  }]
}, {
  "table_name": "measure_unit",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }]
}, {
  "table_name": "acquisition_samples",
  "fields": [{
    "column_name": "fdc_id_of_sample_food",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id_of_acquisition_food",
    "data_type": "INT64"
  }]
}, {
  "table_name": "agricultural_samples",
  "fields": [{
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "acquisition_date",
    "data_type": "DATE"
  }, {
    "column_name": "market_class",
    "data_type": "STRING"
  }, {
    "column_name": "treatment",
    "data_type": "STRING"
  }, {
    "column_name": "state",
    "data_type": "STRING"
  }]
}, {
  "table_name": "sub_sample_result",
  "fields": [{
    "column_name": "food_nutrient_id",
    "data_type": "INT64"
  }, {
    "column_name": "adjusted_amount",
    "data_type": "FLOAT64"
  }, {
    "column_name": "lab_method_id",
    "data_type": "INT64"
  }, {
    "column_name": "nutrient_name",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_attribute_type",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_category",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "code",
    "data_type": "INT64"
  }, {
    "column_name": "description",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_attribute",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "INT64"
  }, {
    "column_name": "seq_num",
    "data_type": "INT64"
  }, {
    "column_name": "food_attribute_type_id",
    "data_type": "INT64"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "value",
    "data_type": "STRING"
  }]
}, {
  "table_name": "food_component",
  "fields": [{
    "column_name": "id",
    "data_type": "INT64"
  }, {
    "column_name": "fdc_id",
    "data_type": "STRING"
  }, {
    "column_name": "name",
    "data_type": "STRING"
  }, {
    "column_name": "pct_weight",
    "data_type": "FLOAT64"
  }, {
    "column_name": "is_refuse",
    "data_type": "BOOL"
  }, {
    "column_name": "gram_weight",
    "data_type": "FLOAT64"
  }, {
    "column_name": "data_points",
    "data_type": "INT64"
  }, {
    "column_name": "min_year_acqured",
    "data_type": "INT64"
  }]
}]
"""

# Instruct the agent to **only** use your dataset
INSTR = f"""
You are a data analysis agent with access to BigQuery tools.
The dataset you have access to contains information from the USDA about foods and nutrician information.
Only query the dataset `{PROJECT_ID}.{DATASET_NAME}`.
Fully qualify every table as `{PROJECT_ID}.{DATASET_NAME}.<table>`.
Never perform DDL/DML; SELECT-only. Return the SQL you ran along with a concise answer.
Here is the database schema, please study it {DB_SCHEMA}
"""

usda_bigquery_agent = Agent(
    model=MODEL,         # Works with ADK; requires a Gemini API key or Vertex AI setup
    name="usda_food_information_bigquery_agent",
    description="""Analyzes tables in a BigQuery dataset that contains food information from the USDA. Tables.""",
    instruction=INSTR,
    tools=[bq_tools],
)

/tmp/ipython-input-1305342453.py:6: UserWarning: [EXPERIMENTAL] BigQueryCredentialsConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  bq_credentials = BigQueryCredentialsConfig(credentials=adc)
/usr/local/lib/python3.11/dist-packages/google/adk/utils/feature_decorator.py:87: UserWarning: [EXPERIMENTAL] BaseGoogleCredentialsConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  return orig_init(self, *args, **kwargs)
/tmp/ipython-input-1305342453.py:9: UserWarning: [EXPERIMENTAL] BigQueryToolConfig: Config defaults may have breaking change in the future.
  bq_tool_cfg = BigQueryToolConfig(write_mode=WriteMode.BLOCKED)
/tmp/ipython-input-1305342453.py:12: UserWarning: [EXPERIMENTAL] BigQueryToolset: This feature is experimental and may change or be removed in future versions without notice. It may

In [ ]:
# @title Define the Allergy Research Agent
import sys, subprocess, importlib, asyncio, re
from urllib.parse import urlparse, urljoin
import httpx
from bs4 import BeautifulSoup


# ---- Configure the site you want to use as the sole information source ----
SITE_URL = "http://www.allergenonline.org/"     # <-- change me
_ALLOWED = urlparse(SITE_URL).netloc.lower()

def fetch_url(url: str) -> dict:
    """
    Fetch and return cleaned text content from a URL within the allowed domain.

    Args:
        url (str): HTTP/HTTPS URL to fetch.

    Returns:
        dict: {
          "url": <final-url>,
          "title": <page title or None>,
          "text": <cleaned text (truncated)>,
          "chars": <len(text)>
        }

    Notes:
      - Only allows URLs on the configured site/domain.
      - Strips scripts/styles and collapses whitespace.
      - Truncates very large pages to keep context small.
    """
    try:
        u = url.strip()
        # Normalize relative links against base, just in case
        if u.startswith("/"):
            u = urljoin(SITE_URL, u)

        parsed = urlparse(u)
        if parsed.scheme not in {"http", "https"}:
            return {"status": "error", "error_message": "Only http/https URLs are allowed."}
        if not parsed.netloc.lower().endswith(_ALLOWED):
            return {"status": "error", "error_message": f"URL not in allowed domain: {_ALLOWED}"}

        # Fetch with sane limits
        with httpx.Client(follow_redirects=True, timeout=20) as client:
            resp = client.get(u, headers={"User-Agent": "adk-site-agent/1.0"})
            resp.raise_for_status()
            ctype = resp.headers.get("content-type", "").lower()
            if "text/html" not in ctype and "application/xhtml" not in ctype:
                return {"status": "error", "error_message": f"Unsupported content-type: {ctype}"}

            soup = BeautifulSoup(resp.text, "html.parser")
            # Remove noise
            for tag in soup(["script", "style", "noscript"]):
                tag.decompose()
            title = (soup.title.string.strip() if soup.title and soup.title.string else None)
            text = re.sub(r"\s+", " ", soup.get_text(separator=" ", strip=True))
            MAX_CHARS = 20000
            if len(text) > MAX_CHARS:
                text = text[:MAX_CHARS] + " ... [truncated]"
            return {"status": "success", "url": str(resp.url), "title": title, "text": text, "chars": len(text)}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}

# ---- Agent that uses Google Search (site-restricted) + fetch_url ----
INSTR = f"""
You are a site-scoped research agent.
Only use Google Search constrained to the domain {_ALLOWED} by prefixing queries with 'site:{_ALLOWED}'.
You research information about food alergens and food health information.
When you identify promising results, call fetch_url(url) to open and read the page content.
Cite the specific URLs you used in your final answer. Do not use sources outside {_ALLOWED}.
"""

allergen_research_agent = Agent(
    model=MODEL,   # or your preferred Gemini model
    name="site_research_agent",
    description=f"Answers questions using only content from {_ALLOWED}.",
    instruction=INSTR,
    tools=[google_search, fetch_url],  # built-in search + custom fetcher
)


In [ ]:
# @title Define the Main (root) Agent
MAIN_AGENT_INSTRUCTIONS="""
You are a friendly food and nutrician agent.
Answer questions related to food, nutrician, allergies, dietary health, and other inquiries related to thise things.
You have 2 helper agents.
- The usda_bigquery_agent has access to a large database from the USDA containing all sorts to food-related information.
- The allergen_research agent can search the Allergen Online web site for allergy-related information.
"""

main_agent = Agent(
    name="main_agent",
    model=MODEL,
    description="Provides Answers to Users Food and Allergy Questions.",
    instruction=MAIN_AGENT_INSTRUCTIONS,
    sub_agents=[usda_bigquery_agent, allergen_research_agent],
)

In [ ]:
# @title Create the Runner that uses our root agent

# --- Session Management ---
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "usda_food_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=main_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='usda_food_app', User='user_1', Session='session_001'
Runner created for agent 'main_agent'.


In [ ]:
# @title Define Agent Interaction Function


async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  return final_response_text

In [ ]:
# @title Run the Initial Conversation

questions = [
    # "List all tables in the dataset.",
    # "Top 10 foods by protein per 100g from the table `food_nutrient` joined with `nutrient`.",
    # "How many rows are in `food`?",
    "Give me a dinner recommendation for someone with allergies to dairy and nuts. Include a meat, starch, and vegetable"
    ]

# We need an async function to await our interaction helper
async def run_conversation():
  for q in questions:
    response = await call_agent_async(query=q,runner=runner,user_id=USER_ID,session_id=SESSION_ID)
    print("-----------------------------")
    print(f"User: {q}")
    print(f"Model: {response}")
    print("-----------------------------")




# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")

-----------------------------
User: Give me a dinner recommendation for someone with allergies to dairy and nuts. Include a meat, starch, and vegetable
Model: Here's a dinner recommendation for someone with dairy and nut allergies:

*   **Meat:** Grilled Chicken Breast
*   **Starch:** Roasted Sweet Potatoes
*   **Vegetable:** Steamed Green Beans

This meal is naturally free of dairy and nuts. When preparing, ensure to use cooking oils that are not nut-based (like olive oil or avocado oil) and check any seasonings or marinades to confirm they are also free from dairy and nuts. Always be mindful of cross-contamination in the kitchen when dealing with severe allergies.
-----------------------------


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f52fd8d3c50>
ERROR:asyncio:Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7f52fd9b6430>, 16609.651991653)])']
connector: <aiohttp.connector.TCPConnector object at 0x7f52fd9858d0>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f52fdb4ec10>
ERROR:asyncio:Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7f52fd9b6270>, 16606.919531088)])']
connector: <aiohttp.connector.TCPConnector object at 0x7f52fe503e90>


In [ ]:
# @title Run a Chat with the Agent.
await chat_with_agent()

Chat started. Type 'exit' (or press Enter on a blank line) to stop.

You: Hi, can you tell me the tables in the FDA dataset?



Agent: The tables in the FDA dataset are: acquisition_samples, agricultural_samples, food, food_attribute, food_attribute_type, food_calorie_conversion_factor, food_category, food_component, food_nutrient, food_nutrient_conversion_factor, food_portion, food_protein_conversion_factor, food_update_log_entry, foundation_food, input_food, lab_method, lab_method_code, lab_method_nutrient, market_acquisition, measure_unit, nutrient, sample_food, sub_sample_food, and sub_sample_result.

You: What foods high in protien?



Agent: Here are some foods that are high in protein, based on the data in the FDA dataset:

| fdc_id  | name     | total_protein |
|---------|----------|---------------|
| 323793  | Protein  | 79.9          |
| 1104705 | Protein  | 51.1          |
| 329490  | Protein  | 48.1          |
| 749420  | Protein  | 40.9          |
| 1104766 | Protein  | 38.6          |
| 329716  | Protein  | 34.2          |
| 331960  | Protein  | 32.1          |
| 335969  | Protein  | 31.6          |
| 335595  | Protein  | 30.8          |
| 335972  | Protein  | 30.5          |

Please note that the fdc_id is a unique identifier for the food in the database. To find out what these foods actually are, you would need to look them up in the `food` table using their `fdc_id`.


You: What foods are high in sugar content?



Agent: Here are some foods that are high in sugar, based on the data in the FDA dataset:

| fdc_id  | name           | total_sugar |
|---------|----------------|-------------|
| 334247  | Sugars, Total  | 99.8        |
| 746784  | Sugars, Total  | 99.8        |
| 746768  | Sugars, Total  | 47.9        |
| 326905  | Sugars, Total  | 47.9        |
| 333008  | Sugars, Total  | 34.8        |
| 747693  | Sugars, Total  | 21.8        |
| 2710843 | Sugars, Total  | 19.16633    |
| 2263890 | Sugars, Total  | 17.341      |
| 2346412 | Sugars, Total  | 17.341      |
| 2346413 | Sugars, Total  | 16.133      |

Please note that the fdc_id is a unique identifier for the food in the database. To find out what these foods actually are, you would need to look them up in the `food` table using their `fdc_id`.


You: Can you show me high protein foods along with their names and provide a brief description?



Agent: Here are some high-protein foods, their descriptions, and protein content per 100g:

| Food Description                                                                               | fdc_id  | protein_per_100g |
|------------------------------------------------------------------------------------------------|---------|-------------------|
| Egg, white, dried                                                                              | 323793  | 79.9              |
| Flour, soy, defatted                                                                           | 1104705 | 51.1              |
| Egg, whole, dried                                                                              | 329490  | 48.1              |
| Pork, cured, bacon, cooked, restaurant                                                         | 749420  | 40.9              |
| Flour, soy, full-fat                                                                           | 1104766 | 38.6              |
| Egg

In [ ]:
# @title Deploy to Agent Engine
from vertexai import agent_engines

# Wrap the agent in an AdkApp object
app = agent_engines.AdkApp(
    agent=main_agent,
    enable_tracing=True,
)

remote_app = agent_engines.create(
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
        "google-auth",
        "google-genai",
        "httpx",
        "beautifulsoup4",
    ]
)

print(f"Deployment finished!")
print(f"Resource Name: {remote_app.resource_name}")
# Resource Name: "projects/{PROJECT_NUMBER}/locations/{LOCATION}/reasoningEngines/{RESOURCE_ID}"

INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.114.0', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.7'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'google-auth', 'google-genai', 'httpx', 'beautifulsoup4', 'cloudpickle==3.1.1', 'pydantic==2.11.7']
INFO:vertexai.agent_engines:Using bucket dsf345gjt
INFO:vertexai.agent_engines:Wrote to gs://dsf345gjt/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://dsf345gjt/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://dsf345gjt/agent_engine/dependencies.tar.gz

For further information visit https://errors.pydantic.dev/2.11/u/class-not-fully-defined
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai

Deployment finished!
Resource Name: projects/569345689354/locations/us-central1/reasoningEngines/412189317067177984
